In [1]:
# intended to sumarize tables of data:
# merged node counts, node types, and drug interactions
# written 11-14-19 JLW

import pickle,os,csv,functools
import pandas as pd
from collections import defaultdict

co_dir = os.path.join('..','data','cotherapy')
cof = os.path.join(co_dir,'co_therapies_with_literature_support.xlsx')

ncf = 'DME_merged_node_counts.xlsx' # this is in the Code directory for now?
tpf = 'drugs_to_dmes_true_positive.txt'

In [2]:
# Read node count file 
# extract intermediate node -> drug relationships
dd_list = functools.partial(defaultdict, list)
all_node_data = {}
df = pd.read_excel(ncf, None)
for dme in df.keys():
    print(dme)
    dme_df = df[dme]
    just_drugs = []
    node_data = defaultdict(dd_list)
    for row in dme_df.itertuples():
        drug_str = row._5
        node_name = row._2
        node_count = row._3
        drug_names = set([dp.split(':')[0] for dp in drug_str.split(';')])
        jd = ','.join(drug_names)
        just_drugs.append(jd)
        node_data[node_name]['count'] = node_count
        node_data[node_name]['drugs'] = jd
    dme_df.insert(1,'Drug Names',just_drugs)
    all_node_data[dme] = node_data

Myopathy
Hemorrhage
Pneumonia
Hypertension
Tardive dyskinesia
Cerebral infarction
Hemolytic anemia
Pancreatitis
Sepsis
Peripheral neuropathy
Gastric ulcer
Hyperlipidemia
Myocardial infarction
Edema
Deep vein thrombosis
Thrombocytopenia
Interstitial lung disease
Pulmonary edema
Delirium
Proteinuria
Ventricular tachycardia
Cardiac arrest
Hepatic necrosis
Agranulocytosis


In [7]:
# open co-therapies table, match co-therapy/node relationships to node-> original drug relationships
dme_mapping = {'DVT':'Deep vein thrombosis', 'ILD':'Interstitial lung disease', 
               'Neuropathy':'Peripheral neuropathy', 'Ventricular Tachy':'Ventricular tachycardia'}
writer = pd.ExcelWriter(open(os.path.join(co_dir,'summary_drug_interactions.xlsx'),'wb'))
df2 = pd.read_excel(cof,None)
for dme in df2.keys():
    print(dme)
    if dme.capitalize() in all_node_data:
        dme_map = dme.capitalize()
    else:
        dme_map = dme_mapping[dme]
    node_data = all_node_data[dme_map]
    just_drugs = []
    node_counts = []
    
    dme_df2 = df2[dme]
    for row in dme_df2.itertuples():
        node_name = row._3
        if ',' in node_name: # sometimes a predicted combination has multiple targets within the DME pathway
            mcnts = []
            mdrugs = []
            for mnode in node_name.split(','):
                [cnt,jd] = [node_data[mnode]['count'],node_data[mnode]['drugs']]
                mcnts.append(str(cnt))
                mdrugs.append(jd)
            just_drugs.append('|'.join(mdrugs))
            node_counts.append('|'.join(mcnts))       
        else: # when a predicted combination has single targets in the dme pathway
            [cnt,jd] = [node_data[node_name]['count'],node_data[node_name]['drugs']]
            just_drugs.append(jd)
            node_counts.append(cnt)
    dme_df2.insert(1,'Drugs with labeled DME',just_drugs)
    dme_df2.insert(1,'Intermediate node count',node_counts)
    
    # reformat
    preferred_cols = ['Drugs with labeled DME','Drug targets in DME pathway','Intermediate node count',
                      'Drug','PMID','Relationship', 'Notes']
    dme_df2 = dme_df2[preferred_cols]
    # dme_df2.rename(columns={'Drug targets in DME pathway': 'DME pathway intermediate', 'Drug': 'Predicted Cotherapy'})
    dme_df2.columns = ['Drugs with labeled DME','DME pathway intermediate','Intermediate node count',
                      'Predicted Cotherapy','PMID','Relationship', 'Notes']
    dme_df2.to_excel(writer,sheet_name=dme)
writer.save()

Cardiac Arrest
Cerebral Infarction
Delirium
DVT
Edema
Gastric Ulcer
Hemorrhage
Hypertension
Hyperlipidemia
ILD
Myocardial Infarction
Myopathy
Neuropathy
Pancreatitis
Pneumonia
Proteinuria
Sepsis
Thrombocytopenia
Ventricular Tachy
